In [ ]:
# | default_exp chat_generator

In [ ]:

# | export


from pathlib import Path
from typing import *
from os import environ
import random
import logging
import time

from fastapi import APIRouter
from pydantic import BaseModel
import openai

/Users/harishm/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
from contextlib import contextmanager
import unittest.mock

import pytest

In [ ]:
# | export

# Reference: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb


def _retry_with_exponential_backoff(
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    max_wait: float = 60,
    errors: tuple = (
        openai.error.RateLimitError,
        openai.error.ServiceUnavailableError,
        openai.error.APIError,
    ),
) -> Callable:
    """Retry a function with exponential backoff."""

    def decorator(func):
        def wrapper(*args, **kwargs):
            num_retries = 0
            delay = initial_delay

            while True:
                try:
                    return func(*args, **kwargs)

                except errors as e:
                    num_retries += 1
                    if num_retries > max_retries:
                        raise Exception(
                            f"Maximum number of retries ({max_retries}) exceeded."
                        )
                    delay = min(
                        delay
                        * exponential_base
                        * (1 + jitter * random.random()),  # nosec
                        max_wait,
                    )
                    logging.info(
                        f"Note: OpenAI's API rate limit reached. Command will automatically retry in {int(delay)} seconds. For more information visit: https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits",
                    )
                    time.sleep(delay)

                except Exception as e:
                    raise e

        return wrapper

    return decorator


@_retry_with_exponential_backoff()
def _completions_with_backoff(*args, **kwargs):
    return openai.ChatCompletion.create(*args, **kwargs)

In [ ]:
# | export

SYSTEM_INSTRUCTION = {
    "role": "system",
    "content": """Your name is Fastkafka AI, an advanced chatbot specialized in Fastkafka. Your primary goal is to assist users to the best of your ability. 
""",
}

In [ ]:
# # | export

# FEW_SHOT_EXAMPLES = [
#     {
#         "role": "system",
#         "name":"example_user",
#         "content": "how to optimise google ads?",
#     },
#     {
#         "role": "system",
#         "name": "example_assistant",
#         "content": """If you're curious, we'd love to show you more! Log in now by clicking the button below and
# discover all the fascinating details. ###show_login_btn###""",
#     },
#     {
#         "role": "system",
#         "name":"example_user",
#         "content": "tallest mountain in the world?",
#     },
#     {
#         "role": "system",
#         "name": "example_assistant",
#         "content": """Unfortunately, I am only capable of providing information related to Google Ads campaigns and their optimization.
#  Is there a specific question or problem you need help with regarding Google Ads? Please let me know, and I'll do my best to help.""",
#     },
#     {
#         "role": "system",
#         "name":"example_user",
#         "content": "can you please tell me a joke?",
#     },
#     {
#         "role": "system",
#         "name": "example_assistant",
#         "content": """Unfortunately, I am only capable of providing information related to Google Ads campaigns and their optimization.
#  Is there a specific question or problem you need help with regarding Google Ads? Please let me know, and I'll do my best to help.""",
#     },
#     {
#         "role": "system",
#         "name":"example_user",
#         "content": "Great job so far, these have been perfect",
#     },
#     {
#         "role": "system",
#         "name": "example_assistant",
#         "content": """Thank you! I'm glad I could assist you. If you have any more questions or need further 
# assistance, feel free to ask.""",
#     },
# ]

In [ ]:
# | export

DEFAULT_MESSAGE_TEMPLATE = [SYSTEM_INSTRUCTION] #+ FEW_SHOT_EXAMPLES

In [ ]:
print(DEFAULT_MESSAGE_TEMPLATE)
assert DEFAULT_MESSAGE_TEMPLATE[0] == SYSTEM_INSTRUCTION
# assert DEFAULT_MESSAGE_TEMPLATE[1:] == FEW_SHOT_EXAMPLES

[{'role': 'system', 'content': 'Your name is Fastkafka AI, an advanced chatbot specialized in Fastkafka. Your primary goal is to assist users to the best of your ability. \n'}]


In [ ]:
messages = DEFAULT_MESSAGE_TEMPLATE + [{"role": "user", "content": "What is a computer?"}]
response = _completions_with_backoff(
    model= "gpt-3.5-turbo",
    messages=messages,
    temperature=0
)

print(response['choices'][0]['message'])

{
  "content": "A computer is an electronic device that can perform various operations on data, such as storing, retrieving, and processing information. It can execute instructions and perform calculations at a high speed, making it a powerful tool for a wide range of applications. Computers come in different forms, including desktops, laptops, tablets, and smartphones, and they are used in various fields, such as education, business, entertainment, and research.",
  "role": "assistant"
}


In [ ]:
@contextmanager
def mock_openai_create():
    mock_choices = {"choices": [{"message": {"content": "This is a mock response"}}]}

    with unittest.mock.patch("openai.ChatCompletion") as mock:
        mock.create.return_value = mock_choices
        yield

In [ ]:
with mock_openai_create():
    response = openai.ChatCompletion.create()
    ret_val = response['choices'][0]['message']['content']
    print(ret_val)
    assert ret_val == "This is a mock response"

This is a mock response


In [ ]:
@_retry_with_exponential_backoff()
def mock_func():
    return "Success"


assert mock_func() == "Success"


# Test max retries exceeded
@_retry_with_exponential_backoff(max_retries=1)
def mock_func_error():
    raise openai.error.RateLimitError


with pytest.raises(Exception) as e:
    mock_func_error()

print(e.value)
assert str(e.value) == "Maximum number of retries (1) exceeded."

Maximum number of retries (1) exceeded.


In [ ]:
# | export

router = APIRouter()


In [ ]:
# | export

class GenerateChatRequest(BaseModel):
    message_history: List[Dict[str, str]]



In [ ]:
# | export


@router.post("/")
def generate_chat_response(
    generate_chat_response_request: GenerateChatRequest,
) -> Dict[str, str]:
    """Generate a chat response.

    Args:
        generate_chat_response_request: A GenerateChatRequest object.

    Returns:
        The response generated by Open Ai's text-davinci-003 model

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    messages = DEFAULT_MESSAGE_TEMPLATE + generate_chat_response_request.message_history
    response = _completions_with_backoff(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    ret_val = {"message": response['choices'][0]['message']['content']}
    return ret_val

In [ ]:
with mock_openai_create():
    message_history = [
        {
            "role": "assistant",
            "content": "How can I help you",
        },
        {"role": "user", "content": "What is your name?"},
    ]
    generate_chat_response_request = GenerateChatRequest(
        message_history=message_history
    )
    actual = generate_chat_response(generate_chat_response_request)
    print(actual)
    assert actual == {"message": "This is a mock response"}

{'message': 'This is a mock response'}
